## Download IPO set from Audit Analytics data from WRDS

-------------------------

### Import libraries

In [1]:
import numpy as np
import pandas as pd
import wrds
pd.options.display.max_columns=200

### Connect WRDS

In [2]:
conn=wrds.Connection()

Enter your WRDS username [yjaey]: yjaeyoon
Enter your password: ···············


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


### Download IPO from Audit Analytics

In [3]:
df= conn.raw_sql(f"""select *
                    from audit.ipo
                    """)
print(df.shape)

(10082, 209)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10082 entries, 0 to 10081
Columns: 209 entries, ipo_info_key to best_edgar_ticker
dtypes: float64(160), object(49)
memory usage: 16.1+ MB


### Data wrangling

In [5]:
# df = df[df.columns.drop(list(df.filter(regex='matchfy')))]
df = df[df.columns.drop(list(df.filter(regex='matchqu')))]
df = df[df.columns.drop(list(df.filter(regex='priorfy')))]
df = df[df.columns.drop(list(df.filter(regex='priorqu')))]
df = df[df.columns.drop(list(df.filter(regex='closestfy')))]
df = df[df.columns.drop(list(df.filter(regex='closestqu')))]
df.shape

(10082, 63)

In [6]:
df.filter(regex=r'matchfy_incmst.*ttm').dropna().head()

,matchfy_incmst_rev_ttm,matchfy_incmst_netinc_ttm,matchfy_incmst_extraitm_ttm,matchfy_incmst_ebitda_ttm
23,9.329600e+09,-1.829000e+08,0.0,626000000.0
46,1.824899e+09,9.410200e+07,0.0,183560000.0
122,7.417702e+09,1.349440e+08,0.0,837309000.0
187,1.812808e+09,1.559632e+09,0.0,0.0
259,1.789776e+09,7.630000e+07,0.0,0.0


In [7]:
df.rename(columns={'matchfy_incmst_rev_ttm':'rev', 'matchfy_incmst_netinc_ttm':'ni'}, inplace= True)
df.head(2)

,ipo_info_key,ipo_date,ipo_date_raw,ipo_name,ipo_tick,ipo_shares,ipo_price,prior_ipo_reg_form,prior_ipo_reg_date,auditor_fkey_at_ipo,aud_name_ipo_date,ipo_desc,accounting_fees,legal_fees,curr_code_fkey,curr_accounting_fees,curr_legal_fees,company_fkey,matchfy_price_date,matchfy_price_close,matchfy_tso_date,matchfy_tso,matchfy_tso_markcap,matchfy_date_qtr,matchfy_date_ttm,matchfy_filing_code_qtr,matchfy_filing_code_ttm,matchfy_annual_quindic,matchfy_balsh_book_val,matchfy_balsh_assets,matchfy_balsh_cash_equivs,matchfy_incmst_rev_qtr,rev,matchfy_incmst_netinc_qtr,ni,matchfy_incmst_extraitm_qtr,matchfy_incmst_extraitm_ttm,matchfy_incmst_ebitda_qtr,matchfy_incmst_ebitda_ttm,matchfy_eff_accchange_qtr,matchfy_eff_accchange_ttm,matchfy_cshflst_op_act_qtr,matchfy_cshflst_op_act_ttm,matchfy_cshflst_inv_act_qtr,matchfy_cshflst_inv_act_ttm,matchfy_cshflst_fin_act_qtr,matchfy_cshflst_fin_act_ttm,matchfy_cshflst_change_qtr,matchfy_cshflst_change_ttm,matchfy_sumfees_fy_end,matchfy_sumfees_fy,matchfy_sum_audfees,matchfy_sum_nonaud,matchfy_sum_benfees,matchfy_sum_itfees,matchfy_sum_taxfees,matchfy_sum_audrel_fees,matchfy_sum_other,matchfy_sum_total,eventdate_aud_fkey,eventdate_aud_name,ipo_type,best_edgar_ticker
0,4161.0,1967-01-01,19670000.0,AAR Corp,AIR,0.0,0.0,None,None,216.0,unknown,None,NaN,NaN,USD,NaN,NaN,0000001750,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216.0,None,None,AIR
1,2677.0,1929-01-01,19290000.0,None,None,0.0,0.0,None,None,216.0,unknown,None,NaN,NaN,USD,NaN,NaN,0000001800,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216.0,None,None,ABT


In [8]:
df=df[['company_fkey', 'ipo_tick', 'ipo_name', 'ipo_date', 'auditor_fkey_at_ipo', 'accounting_fees', 'legal_fees', 'ipo_type', 'rev', 'ni']]
df.dropna(inplace= True)
df.rename(columns={'accounting_fees':'afees', 
                   'legal_fees':'lfees'}, 
          inplace= True)
df['big4']=0
df.loc[df['auditor_fkey_at_ipo']<=4, 'big4']= 1

print(df.shape)
df.head(2)

(3833, 11)


,company_fkey,ipo_tick,ipo_name,ipo_date,auditor_fkey_at_ipo,afees,lfees,ipo_type,rev,ni,big4
72,0000015615,MTZ,Mastec Inc,1997-02-14,1.0,50000.0,100000.0,Traditional,6.594390e+08,34664000.0,1
87,0000018169,DOLE,DOLE FOOD CO INC,2009-10-22,3.0,1500000.0,1000000.0,Traditional,6.778500e+09,132349500.0,1


In [9]:
pd.DataFrame(df.ipo_type.value_counts())

,ipo_type
Traditional,3598
Special Purpose Acquisition Company (SPAC),227
Direct Listing,8


In [10]:
df[df.ipo_type=='Traditional']
df.drop(columns='ipo_type', inplace= True)
df.head()

,company_fkey,ipo_tick,ipo_name,ipo_date,auditor_fkey_at_ipo,afees,lfees,rev,ni,big4
72,0000015615,MTZ,Mastec Inc,1997-02-14,1.0,50000.0,100000.0,6.594390e+08,34664000.0,1
87,0000018169,DOLE,DOLE FOOD CO INC,2009-10-22,3.0,1500000.0,1000000.0,6.778500e+09,132349500.0,1
145,0000029534,DG,Dollar General Corp,2009-11-13,2.0,800000.0,1500000.0,1.179638e+10,339442000.0,1
147,0000029806,CNST,Constar International Inc,2002-11-14,1.0,1200000.0,2250000.0,7.043280e+08,-35388000.0,1
159,0000031347,ELON,Echelon Corp,1998-07-27,5.0,150000.0,300000.0,3.220100e+07,-5851000.0,0


In [11]:
df.head()

,company_fkey,ipo_tick,ipo_name,ipo_date,auditor_fkey_at_ipo,afees,lfees,rev,ni,big4
72,0000015615,MTZ,Mastec Inc,1997-02-14,1.0,50000.0,100000.0,6.594390e+08,34664000.0,1
87,0000018169,DOLE,DOLE FOOD CO INC,2009-10-22,3.0,1500000.0,1000000.0,6.778500e+09,132349500.0,1
145,0000029534,DG,Dollar General Corp,2009-11-13,2.0,800000.0,1500000.0,1.179638e+10,339442000.0,1
147,0000029806,CNST,Constar International Inc,2002-11-14,1.0,1200000.0,2250000.0,7.043280e+08,-35388000.0,1
159,0000031347,ELON,Echelon Corp,1998-07-27,5.0,150000.0,300000.0,3.220100e+07,-5851000.0,0


--------------------

### Save data

In [12]:
df.to_csv('data/aa_ipo.csv', index= False)

### Close connection

In [13]:
conn.close()

-------------